In [24]:
#REFS:
#Wildon, Vertices of Specht Modules and Blocks of the Symmetric Group
#Murphy, The Idempotents of the Symmetric Group and Nakayama's Conjecture
#Murphy, A new construction of Young's seminormal representation of the symmetric groups

In [25]:
SGA_Q3 = SymmetricGroupAlgebra(QQ,3)

In [26]:
SGA_Q3.dft()

[   1    1    1    1    1    1]
[   1  1/2   -1 -1/2 -1/2  1/2]
[   0  3/4    0  3/4 -3/4 -3/4]
[   0    1    0   -1    1   -1]
[   1 -1/2    1 -1/2 -1/2 -1/2]
[   1   -1   -1    1    1   -1]

In [27]:
SGA_F7 = SymmetricGroupAlgebra(GF(7),3)

In [28]:
SGA_F7.dft()

[1 1 1 1 1 1]
[1 4 6 3 3 4]
[0 6 0 6 1 1]
[0 1 0 6 1 6]
[1 3 1 3 3 3]
[1 6 6 1 1 6]

In [29]:
SGA_F7.specht_module(Partition([1,1,1]))

Specht module of [(0, 0), (1, 0), (2, 0)] over Finite Field of size 7

In [30]:
SGA_F3 = SymmetricGroupAlgebra(GF(3),3)

In [31]:
#one cannot perform the DFT when p | n!
try:
    SGA_F3.dft()
except ZeroDivisionError:
    print("Modular case not handled!")

Modular case not handled!


In [32]:
#construct Pierce decomposition of SGA into sum of blocks

In [8]:
#implements modular Fourier transform
#project v onto each block U_i = F_p[S_n]*e_i using \pi_i: v |--> v*e_i as a projection
#this is just a change of basis
def modular_fourier_transform(p,n,nu=1):
    #instantiate group algebra
    SGA_GFp_n = SymmetricGroupAlgebra(GF(p**nu),n)
    #compute the primitive central orthogonal idempotents
    idempotents = SGA_GFp_n.central_orthogonal_idempotents()
    # project v onto each block U_i = F_p[S_n]*e_i via \pi_i: v |--> v*e_i
    B = SGA_GFp_n.basis()
    blocks = [SGA_GFp_n.submodule([b * idem for b in B]) for idem in idempotents]
    # compute the list of basis vectors lifted to the SGA from each block
    block_decomposition_basis = [u.lift() for block in blocks for u in block.basis()]
    # construct the matrix to the standard basis in the order given by the group
    G = SGA_GFp_n.group()
    mat = [[b[g] for b in block_decomposition_basis] for g in G]
    return matrix(SGA_GFp_n.base_ring(), mat)

In [10]:
p=3; n=5

In [11]:
#compute the modular Fourier transform for an example element v
mft = modular_fourier_transform(p,n); mft

120 x 120 dense matrix over Finite Field of size 3 (use the '.str()' method to see the entries)

In [12]:
print(mft.str())

[1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
[0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
[0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
[0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
[0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0

In [61]:
#perform Hill cipher with MFT(p,n) as matrix
txt = "hom encrypt key p"
bit_str = flatten([list(bin_str)[2:] for bin_str in [bin(num) for num in list(txt.encode())]])
encoded_vec = [GF(p)(bit) for bit in bit_str] + (factorial(n)-len(bit_str))*[0]
print(len(encoded_vec))
''.join([str(i) for i in mft*vector(encoded_vec)])

120


'121212121101000010000011200221221010010112100111011212200211222211020010211001101022020020200221221021202022112102211001'

In [62]:
modular_fourier_transform(7,3) == SGA_F7.dft()

False

In [70]:
#note: this matrix has (almost) only has 1's and -1's
M = modular_fourier_transform(3,4)
print(M.str())

[1 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 1 0 0 0 0 0]
[0 1 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 1 0 0 0 0]
[0 0 1 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 1 0 0 0]
[0 0 0 1 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 1 0 0]
[0 0 0 0 1 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 1 0]
[1 2 2 1 1 0 0 0 0 2 2 2 2 2 0 0 0 0 0 0 0 0 0 1]
[0 0 0 0 0 1 0 0 0 0 0 0 0 0 1 0 0 0 0 1 0 0 0 0]
[2 1 0 0 0 1 0 0 0 2 2 0 0 0 2 0 0 0 1 0 0 0 0 0]
[0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 1 0]
[0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 1]
[0 0 2 0 1 0 1 0 0 0 0 2 0 2 0 2 0 0 0 0 1 0 0 0]
[1 2 2 0 1 0 0 1 0 1 1 1 0 1 0 0 2 0 0 0 0 1 0 0]
[0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 1 0 0 0 1 0 0]
[0 0 2 1 0 0 0 0 1 0 0 2 2 0 0 0 0 2 0 0 1 0 0 0]
[1 0 2 0 0 2 1 0 1 2 0 2 0 0 2 2 0 2 0 0 0 0 0 1]
[1 0 2 0 0 2 0 1 1 1 0 1 0 0 1 0 2 1 0 0 0 0 1 0]
[1 2 1 1 1 2 1 0 1 1 1 2 1 1 1 1 0 1 1 0 0 0 0 0]
[1 2 2 0 1 2 0 1 1 2 2 2 0 2 2 0 1 2 0 1 0 0 0 0]
[1 1 2 2 2 1 2 2 2 2 1 2 1 1 1 1 2 1 0 0 0 0 0 1]
[2 0 1 0 2 1 2 2 2 2 0 2 0 2 2 2 1 2 0 0 0 0 1 0]


In [71]:
matrix.identity(6) == modular_fourier_transform(2,3)^4

True

In [72]:
#A fast way for computing the order of 𝑀 is thus 
#to compute the characteristic polynomial P_L of M
#factor it over F_p and check then if prime-divisors of p^k-1
#(for 𝑘 the degree of an involved irreducible polynomial) 
#divide the order
def order_finite_field(M):
    L = M.nrows()
    p = len(M.base_ring())
    P_L = M.charpoly(); P_L
    char_poly_factored = P_L.factor(); char_poly_factored
    degree_list = [item[0].degree() for item in char_poly_factored]
    U = p^(L-1)*prod(p^k-1 for k in degree_list)
    for div in divisors(U):
        if M^div == matrix.identity(L):
            return div

In [ ]:
order_finite_field(mft)

In [55]:
print((M^488488).str())

[1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
[0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
[0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
[0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
[0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
[0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
[0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
[0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
[0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
[0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
[0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0]
[0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0]
[0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0]
[0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0]
[0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0]
[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0]
[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0]
[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0]
[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0]
[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0]


In [ ]:
SymmetricGroupAlgebra(GF(7),3).dft()

In [ ]:
P = Primes()

In [ ]:
P.unrank(4)

In [ ]:
#note: this matrix has (almost) only has 1's and -1's
print(modular_fourier_transform(11,4).str())